## Vigenère Cipher

In our quest for unbreakeable encryption schemes we've tried the shift and the mono alphabetical so far. We've showed that it was easy to break them. Both of the encryption schemes encrypted the same ciphertext for every letter, say, plaintext $e$ will be substituted by $x$ for instance. In the Vigenere cipher a letter in plaintext can be encrypted to several different letters in the ciphertext. This makes the scheme more difficult to crack.

In Vigenere cipher the lenght of the key is arbitrary and indicates the permutation of the text. In the following table I show a simple example: 


|  Variable  |      Value      |
|:----------:|:---------------:|
| Plaintext  | ddddddddddddddd |
| Key        | abc             |
| Ciphertext | defdefdefdefdef |

Here our plaintext is $ddddddddddddddd$ and we encrypt it using the key $abc$ of lenght 3. It is easy to see how it works. The first letter of the plaintext is shifted $a$ (shift of 0), the second shifted $b$ (shift of 1 position) and the third one by $c$ (shift of 3 positions). Therefore $$ d \rightarrow d $$ $$ d \rightarrow e $$ $$ d \rightarrow f $$ 

Another more difficult example is the following:

|  Variable  |            Value            |
|:----------:|:---------------------------:|
| Plaintext  | criptographyisacoolsubject  |
| Key        | esz                         |
| Ciphertext | gjhtlnkjztzxmkzggnpktfbdgl  |

We've chosen as key a random chain of size 3, this is $esz$. Again, $c$ is shifted $e$ (4 positions), $r$ shifted $j$ (9 positions) and $i$ shifted by $h$ (7 positions). Then, the next character $p$ is shifted by the first letter of the key $e$, this results $t$.

Let's code the Vigenere cipher


In [1]:
from random import randint, seed
from copy import deepcopy
import string

seed(1) #fix seed so that we can reproduce the results

characters = string.ascii_lowercase
char_to_num = {c:i for i, c in enumerate(characters)}
num_to_char = {i:c for i, c in enumerate(characters)}

def VigenereKeyGenerator(characters, lenght = 5):
    """Generates a key of fixed lenght
    """
    old_chars = list(deepcopy(characters))
    key = []
    
    while len(key)<lenght:
        elem = old_chars[randint(0,len(old_chars)-1)]
        key.append(elem)
    return ''.join(key)

def ShiftLetter(letter, shiftby, forward=True):
    """Shift a letter a certain ammount of spaces
    letter: an element from string.ascii_lowercase
    shiftby: a letter from string.asccii_lowercase indicating th ammount of shift
    """
    
    listed_chars = list(characters)
    if letter == ' ':
        return letter
    
    shift = char_to_num[shiftby]
    i = char_to_num[letter]
    
    if forward:
        return listed_chars[(i + shift)%len(characters)]
    else:
        return listed_chars[(i - shift)%len(characters)]


def VigenereEncryptDecrypt(text, characters, key, encrypt = True):
    count = 0
    key_len = len(key)
    list_key = list(key)
    
    text2 = ''
    for letter in text:
        c = ShiftLetter(letter, key[count%key_len],forward=encrypt)
        text2 += c
        count+=1
        
    return text2

# generate key
key = VigenereKeyGenerator(characters, 3)

# encrypt a text
sentence = 'criptographyisacoolsubject'
ciphertext = VigenereEncryptDecrypt(sentence, characters, key = key)
plaintext = VigenereEncryptDecrypt(ciphertext, characters, key, encrypt=False)

print("THE KEY: \n{}\n\nTHE SENTENCE:\n{}\n\nCIPHERTEXT:\n{}\n\nPLAINTEXT:\n{}".format(key,sentence,
                                                                                       ciphertext, plaintext))


THE KEY: 
esz

THE SENTENCE:
criptographyisacoolsubject

CIPHERTEXT:
gjhtlnkjztzxmkzggnpktfbdgl

PLAINTEXT:
criptographyisacoolsubject


Obviously the larger the key is, the more difficult it is to decrypt since the space of the keys increases as $26^n$ where $n$ is the lenght of the key. For instance, let's generate a $n=100$ lenght key.

In [2]:
VigenereKeyGenerator(characters, 100)

'ycidpyopumzgdpamntyyawoixzhsdkaaauramvgnxaqhyoprhlhvhyojanrudfuxjdxkxwqnqvgjjspqmsbphxzmnvflrwyvxlco'

It is possible to attack this encryption method if we know the lenght of the key $n$. As in the mono alphabetic encryption we can use the english letters frequency. Let's try it out

In [3]:
def count_char_freqs(text, characters = string.ascii_lowercase):
    freqs = {}
    for letter in characters:
        f = text.count(letter)
        freqs[letter] = f
    return freqs

We download the book 1984 from Orwell and count the characters.

In [4]:
from utils import download_data, process_load_textfile
import string
import os

url = 'http://gutenberg.net.au/ebooks01/0100021.txt'
filename = 'Nineteen-eighty-four_Orwell.txt'
download_path = '/'.join(os.getcwd().split('/')[:-1]) + '/data/'

#download data to specified path
download_data(url, filename, download_path)
#load data and process
data = process_load_textfile(filename, download_path)

#count letters frequency
english_frequencies = count_char_freqs(data)

As before, we consider a random message from the book 1984 to send as a message. The attacker can only see the ciphertext from the message and knows the lenght of the key.

In [5]:
n = round(len(data)*0.05)
i = randint(0, len(data)-1)
sampled_data = data[i:i+n]
encrypted_sampled_data = VigenereEncryptDecrypt(sampled_data, characters, key, encrypt=True)

print("We sample a chunk of {} characters from the book starting at position {}".format(n, i))
print("Using the private key k = {}\n\n".format(key))
print("Sampled Plaintext:")
print(sampled_data)

We sample a chunk of 28471 characters from the book starting at position 533123
Using the private key k = esz


Sampled Plaintext:
ny longer  no he said you dont feel the same  there did not seem to be anything more to say the wind plastered their thin overalls against their bodies almost at once it became embarrassing to sit there in silence besides it was too cold to keep still she said something about catching her tube and stood up to go  we must meet again he said  yes she said we must meet again  he followed irresolutely for a little distance half a pace behind her they did not speak again she did not actually try to shake him off but walked at just such a speed as to prevent his keeping abreast of her he had made up his mind that he would accompany her as far as the tube station but suddenly this process of trailing along in the cold seemed pointless and unbearable he was overwhelmed by a desire not so much to get away from julia as to get back to the chestnut tree cafe which had

Let's write a function for the attacker strategy

In [6]:
def find_key_attack(ciphertext_freqencies, english_frequencies, key_len):
    #now there are a number of key_len different dictionaries to match
    pass